In [1]:
!apt install swi-prolog

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swi-prolog is already the newest version (7.6.4+dfsg-1build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [2]:
!pip install pyswip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 삼각형 개수 구하기

![image.png](attachment:image.png)

In [3]:
from pyswip import Prolog 
 
# Prolog instance 생성 
prolog = Prolog() 

* Describe the picture  

소문자는 하나의 점을 나타냄  

[a, e, k]는 세 점의 리스트를 표현  

line()은 괄호 안의 리스트가 하나의 선임을 나타냄

In [4]:
prolog.assertz("""line([a, e, k])""") 
prolog.assertz("line([a, d, f, j])") 
prolog.assertz("line([a, c, g, i])") 
prolog.assertz("line([a, b, h])") 
prolog.assertz("line([b, c, d, e])") 
prolog.assertz("line([e, f, g, h])") 
prolog.assertz("line([h, i, j, k])")

* Description of the solution   

3개의 line이 존재해야 함 (L1 to L3)  

2개의 line은 반드시 달라야 함. 세 점이 같은 line에 존재하는 경우는 count 하지 않음. (최소 2개의 서로 다른 line이 사용 되는지 check)  

member()는 Prolog predicate. 첫번째 인자가 두번째 인자에 포함되는 경우에 true. 세 점이 서로 다른 line에서 쌍을 이루는지 확인  

“@<”은 세 점이 alphabetical order인 경우에 true. (ex. [a,h,k]와 [a,k,h].) 삼각형이 같은 점을 두 번 또는 세 번 포함하는 경우 제외


In [5]:
prolog.assertz(""" 
triangle(A, B, C) :- 
  line(L1), 
  line(L2), 
  line(L3), 
  L1 \= L2, 
  member(A, L1), 
  member(B, L1), 
  member(A, L2), 
  member(C, L2), 
  member(B, L3), 
  member(C, L3), 
  A @< B,
  B @< C""") 

* Ask Prolog to search for traingles and count them all 

In [6]:
total = 0 
for result in prolog.query("triangle(A, B, C)"): 
  print(result) 
  total += 1 
print("There are", total, "triangles.")

{'A': 'a', 'B': 'e', 'C': 'f'}
{'A': 'a', 'B': 'e', 'C': 'g'}
{'A': 'a', 'B': 'e', 'C': 'h'}
{'A': 'a', 'B': 'd', 'C': 'e'}
{'A': 'a', 'B': 'j', 'C': 'k'}
{'A': 'a', 'B': 'f', 'C': 'g'}
{'A': 'a', 'B': 'f', 'C': 'h'}
{'A': 'a', 'B': 'c', 'C': 'e'}
{'A': 'a', 'B': 'i', 'C': 'k'}
{'A': 'a', 'B': 'c', 'C': 'd'}
{'A': 'a', 'B': 'i', 'C': 'j'}
{'A': 'a', 'B': 'g', 'C': 'h'}
{'A': 'a', 'B': 'b', 'C': 'e'}
{'A': 'a', 'B': 'h', 'C': 'k'}
{'A': 'a', 'B': 'b', 'C': 'd'}
{'A': 'a', 'B': 'h', 'C': 'j'}
{'A': 'a', 'B': 'b', 'C': 'c'}
{'A': 'a', 'B': 'h', 'C': 'i'}
{'A': 'd', 'B': 'e', 'C': 'f'}
{'A': 'c', 'B': 'e', 'C': 'g'}
{'A': 'b', 'B': 'e', 'C': 'h'}
{'A': 'e', 'B': 'h', 'C': 'k'}
{'A': 'f', 'B': 'h', 'C': 'j'}
{'A': 'g', 'B': 'h', 'C': 'i'}
There are 24 triangles.


# 하노이의 탑

```python
def registerForeign(func, name=None, arity=None, flags=0):
    """Register a Python predicate
    ``func``: Function to be registered. The function should return a value in
    ``foreign_t``, ``True`` or ``False``.
    ``name`` : Name of the function. If this value is not used, ``func.func_name``
    should exist.
    ``arity``: Arity (number of arguments) of the function. If this value is not
    used, ``func.arity`` should exist.
    """
```

In [7]:
from __future__ import print_function
from pyswip.prolog import Prolog
from pyswip.easy import getList, registerForeign

N = 3  # Number of disks

def main():
    def notify(t):
        print("move disk from %s pole to %s pole." % tuple(t))
    notify.arity = 1
        
    prolog = Prolog()
    registerForeign(notify)
    prolog.consult("hanoi.pl")
    list(prolog.query("hanoi(%d)" % N))

In [8]:
main()

move disk from left pole to right pole.
move disk from left pole to center pole.
move disk from right pole to center pole.
move disk from left pole to right pole.
move disk from center pole to left pole.
move disk from center pole to right pole.
move disk from left pole to right pole.


In [9]:
from __future__ import print_function
from collections import deque

from pyswip.prolog import Prolog
from pyswip.easy import getList, registerForeign

try:
    input = raw_input
except NameError:
    pass


class Notifier:
    def __init__(self, fun):
        self.fun = fun
        
    def notify(self, t):
        return not self.fun(t)
    notify.arity = 1

In [10]:
class Tower:
    def __init__(self, N=3, interactive=False):
        """N is the number of disks
        """
        self.N = N
        self.disks = dict(left=deque(range(N, 0, -1)), center=deque(), right=deque())
        self.started = False
        self.interactive = interactive
        self.step = 0
        
    def move(self, r):
        if not self.started:
            self.step += 1
            if self.draw():
                return True
            self.started = True
        disks = self.disks
        disks[str(r[1])].append(disks[str(r[0])].pop())
        self.step += 1
        return self.draw()
        
    def draw(self):
        disks = self.disks
        print("\n Step", self.step)
        for i in range(self.N):
            n = self.N - i - 1
            print(" ", end=" ")
            for pole in ["left", "center", "right"]:
                if len(disks[pole]) - n > 0:
                    print(disks[pole][n], end=" ")
                else:
                    print(" ", end=" ")
            print()
        print("-" * 9)
        print(" ", "L", "C", "R")
        if self.interactive:
            cont = input("Press 'n' to finish: ")
            return cont.lower() == "n"

In [11]:
def main():
    N = 3
    INTERACTIVITY = True
    
    prolog = Prolog()
    tower = Tower(N, INTERACTIVITY)
    notifier = Notifier(tower.move)
    registerForeign(notifier.notify)
    prolog.consult("hanoi.pl")
    list(prolog.query("hanoi(%d)" % N))

In [12]:
main()


 Step 1
  1     
  2     
  3     
---------
  L C R
Press 'n' to finish: n


# FWGC

In [13]:
from pyswip import Prolog

from pyswip.core import *
from pyswip import Functor, Variable, Query, call

In [14]:
prolog = Prolog()

# state = Functor("state", 4)

prolog.consult('5-1-FWGC_wo_Print.pl')

In [15]:
list(prolog.query("test"))

[{}]

In [16]:
sol = list(prolog.query('sol(X)'))
sol

[{'X': [Functor(13513229,4,e,e,e,e),
   Functor(13513229,4,w,e,w,e),
   Functor(13513229,4,e,e,w,e),
   Functor(13513229,4,w,e,w,w),
   Functor(13513229,4,e,e,e,w),
   Functor(13513229,4,w,w,e,w),
   Functor(13513229,4,e,w,e,w),
   Functor(13513229,4,w,w,w,w)]}]

In [17]:
for x in sol[0]['X'][::-1]:
    print(x)

state(w, w, w, w)
state(e, w, e, w)
state(w, w, e, w)
state(e, e, e, w)
state(w, e, w, w)
state(e, e, w, e)
state(w, e, w, e)
state(e, e, e, e)


# Pythonic interface (Experimental)

In [18]:
from __future__ import print_function
from pyswip import Functor, Variable, Query, call

assertz = Functor("assertz", 1)
father = Functor("father", 2)
call(assertz(father("michael","john")))
call(assertz(father("michael","gina")))
X = Variable()

In [19]:
q = Query(father("michael",X))
while q.nextSolution():
    print("Hello,", X.value)
q.closeQuery()

Hello, john
Hello, gina
